<a href="https://colab.research.google.com/github/oliverweissl/DSP-Project/blob/main/collab_sft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

# Define Hyperparameters

In [ ]:
model_name = "google/flan-t5-small"
dataset_name = "/content/dataset_complete.csv"
new_model = "flint-flan"
output_dir = "./results"

# Load Datasets and Train

In [ ]:
def _flint_prompt_format(row) -> dict[str,str]:
  """
  Format the prompt automatically.

  :param row: The row conatining the data.
  :return: The prompt.
  """
  with open("/content/eflint.txt", "r") as f:
      context = f.read()

  return {"text": f"""### Instruction:
  Use the Task below and the Input given to write the Response:

  ### Task:
  Understand the concepts in the following text: {context}. Now translate the input into EFlint format.

  ### Input:
  {row['text']}
  ### Response:
  {row["flint"]}
  """}

In [ ]:
dataset = load_dataset('csv', data_files='/content/dataset_complete.csv', split="train")
dataset = dataset.train_test_split(test_size=0.1)

train_mapped = dataset["train"].map(_flint_prompt_format)
val_mapped = dataset["test"].map(_flint_prompt_format)

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_use_double_quant=False,
)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map={"": 0}
)
model.config.use_cache = False
model.config.pretraining_tp = 1

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=2,
    per_device_train_batch_size=4,
    per_device_evaluation_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=25,
    logging_steps=5,
    learning_rate=2e-4,
    weight_decay=0.001,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="all",
    evaluation_strategy="steps",
    eval_steps=5
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=train_mapped,
    eval_dataset=val_mapped,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=None,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=False,
)
trainer.train()
trainer.model.save_pretrained(new_model)